# Excitation Energy Transfer and Decoherence #

Although the molecular exciton model finds application in many different contexts, its perhaps most common application is to the study of excitation energy transfer (EET). EET is the process by which electromagnetic energy (often sunlight) is absorbed by a material and then transported through that material to another, chemically distinct, region. For obvious regions, EET is of great interest the context of photovoltaics and solar light-harvesting, where the focus is usually on creating or investigating materials in which the *rate* of EET is optimized as a function of various material parameters. 

In this exercise, we'll examine the process of EET in a **classical** molecular dimer in the presence of **dynamic disorder**, i.e., environment-induced frequency shifts that change as a function of time. Much like the static disorder (i.e., inhomogeneous broadening) we've seen in our earlier simulations, dynamic disorder leads to dephasing and, in spectroscopic line shape functions, to *homogeneous broadening*. Both static and dynamic disorder play a critical role in EET efficiency. 



## Dynamic Disorder ## 

In real systems, dynamic disorder is a result of low-frequency vibrational motion that leads to a fluctuation of system site energies with time. In our simulation, we won't treat these low-frequency dynamics explicitly, instead just generating a random frequency trajectory for each system oscillator, a treatment strongly analagous to our implicit treatment of the solvent in Langevin dynamics. 

As a simple example, the code below plots a simulated frequency trajectory for a single harmonic oscillator, with parameters typical for an Amide I (peptide C=O stretch) vibration. Note that, although the average frequency across the trajectory is near 1670 cm$^{-1}$, the instantaneous value at each time point varies widely from 1640 cm$^{-1}$ to 1700 cm$^{-1}$. In the next section, we'll examine how this type of dynamic disorder affects energy transfer rates. 

In [ ]:
import math
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import time
import numpy as np
from scipy.optimize import curve_fit
plt.rcParams.update({'font.size': 20})
np.set_printoptions(suppress=True, precision=3)

wo = 6.28*50e12        # Oscillator (angular) frequency
dt=2.0e-15     # Time-step in seconds
Nsteps = 1000
taxis = np.arange(0, Nsteps)*dt
Wx = np.zeros(Nsteps)
Wx[0] = wo
Delta = 1.0e+12
c = 2.9979e+10
for n in range(0,Nsteps-1):
    Wx[n+1] = Wx[n] + np.random.normal(0, Delta, (1)) + 0.01*(wo-Wx[n])

    
plt.plot(taxis*1e+15, Wx/(2.0*math.pi*c))
plt.xlabel('Time (fs)')
plt.ylabel('Frequency (cm$^{-1}$)')
plt.show()

## EET in a Coupled Dimer ##

As a simple test-system for exploring EET dynamics, we'll consider a dimer of two oscillators with perpendicular dipole moments, which interact (e.g., via dipole-dipole coupling) with a coupling strength $J$. To simulate the EET process, we'll excited the system with a laser pulse whose polarization is parallel to **one** of the two dipole moments and orthogonal to the other. Thus the initial excitation created by the pulse is localized exclusively on Oscillator 1 -- the **donor** -- but can be transferred over time to Oscillator 2 -- the **acceptor**. Because our simulation scheme relies on random oscillator frequencies, we'll need to ensemble-average over many different randomly-sampled trajectories, reporting at the end only the ensemble-averaged quantities that are relevant to macroscopic measurements. 


The code below defines two functions, ``calc_response()`` and ``fit_data()``. The first function simulates the actual interaction of our dipole system with an ultrafast laser pulse and returns a matrix ``E`` whose columns store the energy of the donor (first column) and acceptor (second column) as a function of simulation time. The second function fits the energy profile of the donor to a functional form consisting of an exponential decay multiplied by a cosine signal. As we'll see, this form is flexible enough to provide a reasonable description of the EET dynamics over a wide range of system parameters. 

Note that the "EET time" generated by the code is the time-scale for the exponential decay process (reflecting energy equilibration between the two oscillators), **not** than the time-scale of the cosine signal that reflects rapid oscillation of energy back and forth between donor and acceptor. In the context of solar light-harvesting, it is usually the exponential time scale that is of interest, since energy must remain localized on the acceptor long enough for the "next step" in the energy transfer process to take place (e.g., transfer to another molecule, or a chemical event like charge separation). 

Finally, it is also noteworthy that, since our simulation is classical, the long-time limit respects the classical *equipartition principle*, meaning that as the system equilibrates the excitation energy will ultimately be shared equally between the two oscillators. **The key distinction between our classical simulation and a quantum energy-transfer process is that quantum statistics violate the equipartition principle** -- so that EET in a quantum system can result in the localization of energy on low-frequency oscillators. This principle has important consequences in solar light-harvesting, as evidenced by the "energy funnel" structure apparent in biological light-harvesting complexes, where low-frequency pigments are concentrated near the "reaction center" where charge-separation takes place. 

After browsing through the code, execute the cell and then play with the simulation values by modifying and executing the code under the next heading. 

In [ ]:
import math
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import time
import numpy as np
from scipy.optimize import curve_fit
plt.rcParams.update({'font.size': 20})
np.set_printoptions(suppress=True, precision=3)

def calc_accel(x,efield, wx, J):
    K = M*wx**2
    ax = -K*x/M
    ax += Qo*efield*Minv*DipMat[:,0]  # Excitation is along Z-axis: so we use DipMat[:,0]
    ax[:,0] += -J*x[:,1]*Minv
    ax[:,1] += -J*x[:,0]*Minv 
    return ax

def vv_step(x,vx,ax,efield,wx,J):
    if(Temp==0):
        axrand = 0
    else:
        axrand = math.sqrt(2.0*kB*Temp*gamma/dt)*np.random.normal(0,1,1)/M
    xnew = x + B*dt*vx + 0.5*B*dt*dt*(ax + axrand)
    axnew = calc_accel(xnew,efield, wx,J)
    vxnew = A*vx + 0.5*dt*(A*ax + axnew + 2.0*B*axrand)
    return xnew,vxnew,axnew

def gauss_pulse(t):
    return np.cos(2.0*math.pi*(t-to)*nu)*np.exp(-((t-to)**2)/(2.0*sigma*sigma))

to = 150e-15
sigma = 10e-15
nu = 50e+12

tmax = 10e-12      # Total simulation time in seconds
dt=2.0e-15     # Time-step in seconds
Nsteps=int(tmax/dt)
M=12*(1.66054e-24)   # Mass in g
Minv = 1.0/M
Qo = 4.803e-10         # Elementary charge in statCoulombs
taxis = np.arange(0,Nsteps)*dt   # Time axis (array of time steps)

gamma = 0.0e-12  # grams/second
hbar=1.0546e-27
c = 2.9979e+10
kB = 1.38064852e-16                  # erg/K
Temp = 0.0                           # K
B = 1.0/(1.0 + 0.5*gamma*dt/M)
A = B*(1.0 - 0.5*gamma*dt/M)

Nsamples = 2000
Nosc = 2
DipMat = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])

wo = 6.28*50e12        # Oscillator (angular) frequency

vaxis = np.fft.fftfreq(Nsteps)/(dt*c)

v1 = 1600
v2 = 1750
ndx1 = np.where(np.abs(vaxis-v1)==np.min(np.abs(vaxis-v1)))[0][0]
ndx2 = np.where(np.abs(vaxis-v2)==np.min(np.abs(vaxis-v2)))[0][0]


Emax = 50e+4
Efield = Emax*gauss_pulse(taxis)
def calc_response(JNorm, DeltaNorm, showflag):
    
    Delta = 1.0e+12*DeltaNorm
    J = 10e+3*JNorm
    Wo = wo*np.ones((Nsamples,Nosc))
    Wx = Wo
    X = np.zeros((Nsamples,Nosc))
    VX = np.zeros((Nsamples,Nosc))
    AX = calc_accel(X, Efield[0], Wx, J)
    MuY = np.zeros((Nsteps))
    MuZ = np.zeros((Nsteps))
    E = np.zeros((Nsteps,Nosc))
    #Wx = np.random.normal(wo, 0.1*wo, (Nsamples,Nosc))
    Wtraj = np.zeros((Nsteps,Nosc))
    for n in range(0,Nsteps):
        Wx = Wx + np.random.normal(0, Delta, (Nsamples,Nosc)) + 0.01*(wo-Wx)
        Wtraj[n,:] = Wx[0,:]
        X,VX,AX = vv_step(X,VX,AX,Efield[n],Wx,J)
        MuY[n] = np.mean(X*DipMat[:,1])*Qo
        MuZ[n] = np.mean(X*DipMat[:,0])*Qo
        E[n,:] = 0.5*M*np.mean(Wx*Wx*X*X + VX*VX,0)
    
    if(showflag):
        maxE = np.max(E[:,0]+E[:,1])
        
        plt.figure(figsize=(10,3))    
        plt.plot(taxis*1e+15, Wtraj[:,0]*1e-12, 'r', label='Donor')
        plt.plot(taxis*1e+15, Wtraj[:,1]*1e-12, 'b', label='Acceptor')
        plt.ylabel('$Frequency$ (cm$^{-1}$)')
        plt.legend(loc = 'upper right')
        plt.title('Frequency Trajectories')

        plt.figure(figsize=(10,3))
        plt.plot(taxis*1e+15, E[:,0]/maxE, 'r', label='$E_{donor}$')
        plt.plot(taxis*1e+15, E[:,1]/maxE, 'b', label='$E_{acceptor}$')
        plt.plot(taxis*1e+15, (E[:,0]+E[:,1])/maxE, 'k', label='$E_{total}$')
        plt.legend(loc='upper right')
        plt.title('Energy Profiles')
        plt.show()

    return E,Wtraj


def fit_data(E, showflag):
    maxE = np.max(E[:,0]+E[:,1])
    ndxo = np.where(np.min(np.abs(taxis-(to+5.0*sigma)))==np.abs(taxis-(to+5.0*sigma)))[0][0]

    xdata = (taxis[ndxo:]-taxis[ndxo])*1e+12
    ydata = E[ndxo:,0]/maxE

    def func(x, a, b, d):
        return a * np.exp(-b * x)*np.cos(2.0*math.pi*d*x) + 0.5

    guess = [0.5, 1.0, 5.0]
    popt, pcov = curve_fit(func, xdata, ydata, guess)

    if(showflag):
        plt.figure(figsize=(8,6))
        plt.plot(xdata, ydata, 'k-', label='Raw data', linewidth=3)
        plt.plot(xdata, func(xdata, *popt), 'r-', label='Fit: $\\tau_{EET} =$'+str(round((1.0/popt[1])*100)/100)+' ps', linewidth=3)
        plt.plot(xdata, func(xdata, popt[0], popt[1], 0.0), 'b--', label='Exponential Component', linewidth=3)
        plt.title('Fit Results')
        plt.legend()
        plt.show()
    
    return 1.0/popt[1]

## EET Dynamics ##

To get a sense for how the simulation works, execute the code block below, playing with the values of the parameters ``Delta`` and ``J``. ``Delta`` controls the strength of dynamic disorder -- i.e., the range of the frequency fluctuations for each oscillator -- while ``J`` controls the interaction strength between the two sites. Both parameters are normalized, so that values of ``J = Delta = 1`` represents a "reasonable" set of parameters, comparable to what one might see in a real Amide I experiment. 

Note that the last argument in the two functions ``calc_response()`` and ``fit_data()`` is a boolean flag which indicates whether or not the results should be plotted. When set to ``1`` (the default), the generated data is plotted graphically. When set to ``0``, the simulation runs but no figures are generated. 

To see how the EET rate depends on the value of the two parameters ``J`` and ``Delta``, execute the code below, starting with ``J = Delta = 1`` and then with a variety of parameter values, noting how each variable effects the EET time scale, reported as $\tau_{EET}$ in the last plot. 

In [ ]:
J = 1.0
Delta = 1.0

E,Wtraj = calc_response(J, Delta, 1)
tau = fit_data(E, 1)

## Variation of EET Rate with Site-to-Site Coupling ##

Now that you have some feel for how the simulation works, let's see how the overall EET rate (one over the EET time) varies with the value of $J$. The code below repeats the EET simulation for values of $J$ ranging from one-tenth to ten-times our "normal" value. How does the EET rate vary with J? Why might this be? 

In [ ]:
Delta = 1.0 
Jvec = np.arange(0.25, 10.0, 0.5)

Delta = 1.0
Tau = np.zeros(np.shape(Jvec))
for n in range(0, len(Jvec)):
    plt.close('all')
    E,Wtraj = calc_response(Jvec[n], Delta, 0)
    Tau[n] = fit_data(E, 0)

plt.plot(Jvec, 1.0/Tau, 'o-', label='$k_{ET}$')
plt.xlabel('Coupling Strength (arb. units)')
plt.ylabel('$k_{ET}$ (ps$^{-1}$)')
plt.legend()
plt.show()

## Variation of EET Rate with Dynamic Disorder ##

Now that you have some idea how the EET rate varies with ``J``, let's study more closely its variation with ``Delta``. The cell below repeats our simulation scheme with ``J`` fixed and ``Delta`` varying from one-tenth to twice our "standard" value. (Note that the simulation becomes unstable when ``Delta`` is made too big, which is why we explore a more restricted range of values than with ``J``. How does the EET rate vary with ``Delta``? Why might this be?

In [ ]:
J = 1.0
Dvec = np.arange(0.1, 2.0, 0.1)
Tau = np.zeros(np.shape(Dvec))
for n in range(0, len(Dvec)):
    E,Wtraj = calc_response(J, Dvec[n], 0)
    Tau[n] = fit_data(E, 0)

plt.plot(Dvec, 1.0/Tau, 'o-', label='$k_{ET}$')
plt.xlabel('Dephasing Strength')
plt.ylabel('$k_{ET}$ (ps$^{-1}$)')
plt.legend()
plt.show()